# Import & global vars

In [1]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard

import os
import datetime
import pandas as pd
import numpy as np
from ast import literal_eval

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
data_path = ''

# Data loading

In [3]:
qna_tags = pd.read_csv(data_path + 'merged_data.csv')

w2v_train = np.load(data_path + 'doc2vec_train.npy', allow_pickle=True)
w2v_test = np.load(data_path + 'doc2vec_test.npy', allow_pickle=True)

X_train = pd.DataFrame(w2v_train, columns=['A', 'Q'])
X_test = pd.DataFrame(w2v_test, columns=['A', 'Q'])


In [4]:
X_train.head()

,A,Q
0,"[0.29619655, 0.13556147, -0.054417618, 0.05035...","[0.1957434, 0.076143995, -0.1938008, -0.216860..."
1,"[0.046823636, -0.14547096, -0.23043968, -0.147...","[0.13528667, 0.13512334, -0.22924286, -0.17696..."
2,"[0.23061377, 0.108394705, -0.015458137, -0.056...","[0.22953051, 0.10089784, -0.22944072, -0.16675..."
3,"[0.5029813, -0.11446538, -0.15049075, -0.12557...","[0.2499625, 0.10600598, -0.20118609, -0.166990..."
4,"[0.39167732, -0.2809122, 0.105671346, 0.147031...","[0.25080028, 0.0806821, 0.0057426286, -0.13059..."


# Data standardization

In [5]:
qna_tags = qna_tags[:25000]
ref_time = datetime.datetime(2008, 1, 1)

def to_integer(dt_timeQ, dt_timeA):
    timeQ = dt_timeQ - ref_time
    timeA = dt_timeA - ref_time
    time_diff = 1 - (timeQ.days*86400 + timeQ.seconds) / (timeA.days*86400 + timeA.seconds)
    return time_diff

    
def get_time_diff(df):
    for index, row in df.iterrows():
        row.CreationDate_A = datetime.datetime.strptime(row.CreationDate_A, "%Y-%m-%dT%H:%M:%SZ")
        row.CreationDate_Q = datetime.datetime.strptime(row.CreationDate_Q, "%Y-%m-%dT%H:%M:%SZ")
        qna_tags.at[index, 'time_diff'] = to_integer(row.CreationDate_Q, row.CreationDate_A)
    return df


qna_tags = get_time_diff(qna_tags)
qna_tags['score'] = qna_tags['Score_A'] + qna_tags['Score_A'] * qna_tags['time_diff']

y_test = qna_tags[12500:]
y_train = qna_tags[:12500]


In [6]:
#y_train = y_train['score']
#y_test = y_test['score']
print(X_train.loc[0, 'A'].size)

300


# Model building

In [30]:
from keras_self_attention import SeqSelfAttention

#LSTM model
def build_LSTM_model(timesteps, features, h1=256):
    print('Build model...')
    
    model = Sequential()
    
    model = Sequential()
    model.add(LSTM(256, return_sequences=True, stateful=True, 
                   batch_input_shape=(batch_size, timesteps, features)))
    model.add(LSTM(128,stateful=True, return_sequences=True))
    model.add(LSTM(128,stateful=True, return_sequences=True))
    model.add(LSTM(64,stateful=True))
    model.add(tf.keras.layers.SeqSelfAttention(attention_width=cfg.ATTNWIDTH,
                attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                attention_activation='softmax',
                name='Attention'))
    model.add(Dense(1, activation='linear'))   
    return model

Using TensorFlow backend.


ImportError: cannot import name 'transpose_shape'

In [9]:
#Attention model
#def build_focus_model:
#    return 0

SyntaxError: invalid syntax (<ipython-input-9-63d7f315d32d>, line 2)

# Model compiling

In [9]:
def compile_model(model, loss='mean_squared_error', optimizer='adam'):
    model.compile(loss=loss,
                  optimizer=optimizer,
                  metrics=['MSE']
                  )
    return  model

# Model fitting

In [17]:
batch_size = 50

In [18]:
def fit_model( x_train, y_train, model, batch_size,  epochs = 5):
    os.makedirs("./logs",exist_ok=True)
    tensorboard = TensorBoard(log_dir=os.path.join('./logs'), histogram_freq=0,
                              write_graph=True, write_images=False,profile_batch = 100000000)
    Es= EarlyStopping(monitor='val_loss', patience=2)
    callbacks = [ Es, tensorboard]
    
    print('Train...')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              callbacks= callbacks)
    return model

# Predicting

In [19]:
def predictBestAnswer(model, question, answers):
    bst_score = 0
    for answer in answers:
        ans_score = model.predict(question, answer)
        if ans_score > bst_score:
            bst_score = ans_score
    return bst_score

In [13]:
def to_np()

SyntaxError: invalid syntax (<ipython-input-13-9cf8bb0bc4ab>, line 1)

# Main training

In [14]:
# Put your inputs into a single list
X_train['single_input_vector'] = X_train[['A', 'Q']].apply(tuple, axis=1).apply(list)
# Double-encapsulate list so that you can sum it in the next step and keep time steps as separate elements
X_train['single_input_vector'] = X_train.single_input_vector.apply(lambda x: [list(x)])
# Use .cumsum() to include previous row vectors in the current row list of vectors
X_train['cumulative_input_vectors'] = X_train.single_input_vector.cumsum()

y_train['output_vector'] = y_train[['score']].apply(tuple, axis=1).apply(list)


# Pad your sequences so they are the same length
#from keras.preprocessing.sequence import pad_sequences

#max_sequence_length = X_train.cumulative_input_vectors.apply(len).max()
# Save it as a list   
#padded_sequences = pad_sequences(X_train.cumulative_input_vectors.tolist(), max_sequence_length).tolist()
#X_train['padded_input_vectors'] = pd.Series(padded_sequences).apply(np.asarray)

# Extract your training data
X_train_init = np.asarray(X_train.single_input_vector)
# Use hstack to and reshape to make the inputs a 3d vector
X_train = np.hstack(X_train_init).reshape(len(X_train),300,len(['A', 'Q']))
y_train = np.hstack(np.asarray(y_train.output_vector)).reshape(len(y_train),len(['score']))


/home/valentin/.conda/envs/majeur_ml/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [93]:
print(X_train)
print(y_train)

[[[ 2.96196550e-01  1.35561466e-01]
  [-5.44176176e-02  5.03530167e-02]
  [ 9.71140191e-02  3.17598780e-04]
  ...
  [ 1.09442547e-01 -2.39401907e-01]
  [-3.39948982e-01  7.52227157e-02]
  [-1.46377519e-01 -3.56734335e-01]]

 [[ 4.68236357e-02 -1.45470962e-01]
  [-2.30439678e-01 -1.47461593e-01]
  [ 2.21834145e-02 -1.21305682e-01]
  ...
  [ 1.00411445e-01 -1.99947163e-01]
  [-3.67973626e-01  1.03055134e-01]
  [-2.02707380e-01 -3.82397860e-01]]

 [[ 2.30613768e-01  1.08394705e-01]
  [-1.54581368e-02 -5.66973090e-02]
  [-1.51115947e-03 -1.23994745e-01]
  ...
  [ 5.44580966e-02 -2.83200800e-01]
  [-3.86532843e-01  6.26603216e-02]
  [-2.05040678e-01 -2.93880254e-01]]

 ...

 [[ 2.27501169e-01  5.01157269e-02]
  [-9.57589522e-02 -1.70819074e-01]
  [ 3.05057973e-01 -2.64334381e-01]
  ...
  [ 1.63045913e-01  3.99874337e-02]
  [-5.50860524e-01 -1.37738725e-02]
  [-4.85165231e-03 -2.09943280e-01]]

 [[ 2.07316548e-01 -2.89015949e-01]
  [-4.81763110e-02 -3.99295725e-02]
  [ 2.09155697e-02 -6.2699

In [28]:
import tensorflow as tf

with tf.device("/CPU:0"):
    model = build_LSTM_model(X_train.shape[1], X_train.shape[2], h1=512)
    model = compile_model(model)
    model = fit_model(X_train, y_train, model, batch_size, epochs = 10)

Build model...


AttributeError: module 'tensorflow.keras.layers' has no attribute 'SeqSelfAttention'